In [30]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [31]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        # Most common price format
        price = soup.find("span", {"class": "a-price-whole"}).text
        symbol = soup.find("span", {"class": "a-price-symbol"}).text
        return f"{symbol}{price}".strip()

    except AttributeError:
        try:
            # Sometimes Amazon shows price inside another span
            price = soup.find("span", {"class": "a-offscreen"}).text
            return price.strip()
        except:
            return ""

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [ ]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=mars+lipstick+set&crid=2343OT53Z8LMY&sprefix=mars+%2Caps%2C342&ref=nb_sb_ss_mvt-t11-ranker_1_5"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [33]:
amazon_df

,title,price,rating,reviews,availability
0,MARS Matte Box Set of 3 Lipsticks for Women | ...,₹317.,4.1 out of 5 stars,"6,331 ratings",In stock
1,MARS Creamy Matte Long Lasting Lipstick for Wo...,₹177.,4.1 out of 5 stars,759 ratings,In stock
2,MARS Matte Muse Mousse Lipstick (3ml) (01-LEAD...,₹269.,4.0 out of 5 stars,789 ratings,In stock
3,LoveChild Masaba - Paan-Tastic - Luxe Matte Li...,₹520.,4.2 out of 5 stars,879 ratings,In stock
4,SUGAR POP Matte Ultrastay Lipstick | Vitamin E...,₹160.,4.0 out of 5 stars,"1,775 ratings",In stock
5,Swiss Beauty Pure Lustre Lipstick | High Shine...,₹316.,4.4 out of 5 stars,163 ratings,In stock
6,"RENEE Very Matte - Pack of 4 Matte Lipsticks, ...",₹449.,4.0 out of 5 stars,"3,054 ratings",In stock
7,LoveChild Masaba Pocket Mini Lipsticks Set of ...,₹999.,4.4 out of 5 stars,377 ratings,In stock
8,MARS Queen of Mattes Liquid Lipsticks Box Set ...,₹434.,4.3 out of 5 stars,122 ratings,In stock
9,MARS Long Lasting Crayon Lipstick up to 12 Hou...,₹248.,4.0 out of 5 stars,"5,344 ratings",In stock


In [ ]:
amazon_df.to_excel("amazon_data.xlsx", index=False)